<a href="https://colab.research.google.com/github/choureymanas/Stack-Overflow-Question-Tagging-/blob/main/StackOverflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color = 'blue'> **NN with pre_trained Embeddings**


<font color = 'green'>  **Objective:**
- Learn to use pre-trained embeddings in a Neural Network</font>

# <font color = 'blue'> **Specify Project Folders**

In [ ]:
from pathlib import Path

In [ ]:
custom_functions = '/content/drive/MyDrive/custom_functions'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys

sys.path.append(str(custom_functions))

In [ ]:
sys.path

['/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/drive/MyDrive/custom_functions']

# <font color = 'blue'>  Import libraries

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
if 'google.colab' in str(get_ipython()):
  !pip install --upgrade gensim -qq

     |████████████████████████████████| 24.1 MB 1.4 MB/s 


In [ ]:
if 'google.colab' in str(get_ipython()):
  !pip install wandb --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 9.2 MB/s 
     |████████████████████████████████| 182 kB 49.5 MB/s 
     |████████████████████████████████| 168 kB 50.9 MB/s 
     |████████████████████████████████| 62 kB 1.2 MB/s 
     |████████████████████████████████| 168 kB 52.2 MB/s 
     |████████████████████████████████| 166 kB 53.3 MB/s 
     |████████████████████████████████| 166 kB 24.6 MB/s 
     |████████████████████████████████| 162 kB 27.5 MB/s 
     |████████████████████████████████| 162 kB 51.5 MB/s 
     |████████████████████████████████| 158 kB 53.4 MB/s 
     |████████████████████████████████| 157 kB 47.9 MB/s 
     |████████████████████████████████| 157 kB 60.9 MB/s 
     |████████████████████████████████| 157 kB 29.1 MB/s 
     |████████████████████████████████| 157 kB 61.0 MB/s 
     |████████████████████████████████| 157 kB 54.4 MB/s 
     |████████████████████████████

In [ ]:
# Import random function

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext.vocab import  vocab
from torch.optim.lr_scheduler import ReduceLROnPlateau, OneCycleLR, StepLR

import wandb
import spacy
#import custom_preprocessor as cp

import random
from datetime import datetime
import numpy as np
from pathlib import Path
import pandas as pd
import joblib
from collections import Counter

from pathlib import Path

from sklearn.model_selection import train_test_split

import gensim
from gensim.models import KeyedVectors

from types import SimpleNamespace

from sklearn.preprocessing import MultiLabelBinarizer as mlb

In [ ]:
gensim.__version__

'4.2.0'

In [ ]:
data_folder = '/content/drive/MyDrive/HW 6/multilabel_hw.csv'
save_model_folder = '/content/drive/MyDrive/HW 6 B/Models'

We will be using W&B for visualization.

In [ ]:
# Login to W&B
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## <font color = 'blue'> Train/Test/Valid Dataset

In [ ]:
data = pd.read_csv(data_folder)
data_sample = data.sample(frac =0.1)

In [ ]:
# creating Pandas Dataframe
# train_data = pd.read_csv(train_file, index_col=0)
# test_data = pd.read_csv(test_file, index_col=0)
train_data = data_sample

In [ ]:
# print shape of the datasets
print(f'Shape of Training data set is : {train_data.shape}')
#print(f'Shape of Test data set is : {test_data.shape}')

Shape of Training data set is : (4743, 7)


# Target Variable

In [ ]:
!pip install swifter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 830 kB 7.1 MB/s 
     |████████████████████████████████| 280 kB 53.9 MB/s 
     |████████████████████████████████| 1.6 MB 46.2 MB/s 
  Created wheel for swifter: filename=swifter-1.3.4-py3-none-any.whl size=16321 sha256=84637183df24c93aa4a4e3ae3fdf6ac642cabaeab146fa136fdc570bc0bad6ee
  Stored in directory: /root/.cache/pip/wheels/29/a7/0e/3a8f17ac69d759e1e93647114bc9bdc95957e5b0cbfd405205
Successfully built swifter
  Attempting uninstall: psutil
    Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfully uninstalled psutil-5.4.8


In [ ]:
import swifter
import ast 
train_data['Tag_Number_list'] = train_data['Tag_Number'].apply(lambda x : ast.literal_eval(x))

[autoreload of psutil failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/usr/local/lib/python3.7/dist-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/usr/lib/python3.7/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/usr/lib/python3.7/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 728, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/usr/local/lib/python3.7/dist-packages/psutil/__init__.py", line 83, in <module>
    from ._common import AccessDenied
ImportError: cannot import name 'AccessDenied' from 'psutil._common' (/usr/local/lib/python3.7/dist-packages/psutil/_

In [ ]:
y_final = train_data['Tag_Number_list'].values
y_one_hot = mlb().fit_transform(y_final)

In [ ]:
train_data.head()

,Unnamed: 0,Unnamed: 0.1,Id,Title,Body,Tags,Tag_Number,Tag_Number_list
10923,49242,5008439,5008440,Creating visual designs for android applications,<p>I've been on and off with Android developme...,java android,"[1, 4]","[1, 4]"
13341,38107,3878300,3878301,How to stop the finish() method after press ba...,<p>Can I check some condition in finish()</p>\...,java android,"[1, 4]","[1, 4]"
9938,6248,633054,633055,How can I prevent button-mashing when using an...,<p>I'm using buttons to submit data via Ajax. ...,javascript jquery,"[3, 5]","[3, 5]"
40594,7225,728347,728348,Store the text entered by the user as it is in...,<p>How to store the text entered by the user a...,c# asp.net,"[0, 9]","[0, 9]"
11577,56212,5719613,5719614,Is there a man signature generator in jquery?,<p>Is there a plugin on jquery which you can c...,php jquery,"[2, 5]","[2, 5]"


In [ ]:
X = train_data['Body'].values

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y_one_hot, test_size = 0.20, random_state=42)

In [ ]:
X_test , y_test = train_data['Body'].values, y_one_hot

## <font color = 'blue'>  Data PreProcessing

<font color = 'green'> We will use the preprocessed files we created earlier.

In [ ]:
import custom_preprocessor as cp
#X_train_cleaned = cp.SpacyPreprocessor(model = 'en_core_web_sm', batch_size=1000).transform(X_train)

In [ ]:
#X_valid_cleaned = cp.SpacyPreprocessor(model = 'en_core_web_sm', batch_size=1000).transform(X_valid)
#X_test_cleaned = cp.SpacyPreprocessor(model = 'en_core_web_sm', batch_size=1000).transform(X_test)

In [ ]:
# X_train_cleaned_file = data_folder / 'x_train_cleaned_bag_of_words.pkl'
# X_valid_cleaned_file = data_folder / 'x_valid_cleaned_bag_of_words.pkl'
# X_test_cleaned_file = data_folder / 'x_test_cleaned_bag_of_words.pkl'

X_train_cleaned_file = '/content/drive/MyDrive/HW 6 B/x_train_cleaned_bag_of_words.pkl'
X_valid_cleaned_file =  '/content/drive/MyDrive/HW 6 B/x_valid_cleaned_bag_of_words.pkl'
X_test_cleaned_file =  '/content/drive/MyDrive/HW 6 B/x_test_cleaned_bag_of_words.pkl'

In [ ]:
# joblib.dump(X_train_cleaned, X_train_cleaned_file)
# joblib.dump(X_valid_cleaned, X_valid_cleaned_file)
# joblib.dump(X_test_cleaned, X_test_cleaned_file)

In [ ]:
X_train_cleaned = joblib.load(X_train_cleaned_file)
X_valid_cleaned = joblib.load(X_valid_cleaned_file)
X_test_cleaned = joblib.load(X_test_cleaned_file)

In [ ]:
print(type(X_train_cleaned))
print(type(y_train))

<class 'list'>
<class 'numpy.ndarray'>


## <font color = 'blue'> Custom Dataset Class

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    """IMDB dataset."""

    def __init__(self, X, y):
        self.X = np.array(X)
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        text = self.X[idx]
        labels = self.y[idx]
        sample = (text, labels)
        
        return sample

In [ ]:
trainset = CustomDataset(X_train_cleaned,y_train)
validset = CustomDataset(X_valid_cleaned,y_valid)
testset = CustomDataset(X_test_cleaned,y_test)

In [ ]:
trainset.__getitem__([11])

(array(['array variable value textfields var datum = address city country   command   transaction.executesql("insert customer(name address city country values data[0 data[1 data[2 data[3   work fine real database 50 field need add example add question mark data[n error prone think create loop instead array var datum = address city country   loop var column = var questionmark = var = 0 < i++      column + = data + +      questionmark + =   column = columns.slice(0 -2 questionmark = questionmarks.slice(0 -2 column = + column +   use questionmark command like transaction.executesql("insert customer(name address city country values + questionmark + data[0 data[1 data[2 data[3   try use column work transaction.executesql("insert customer(name address city country values + questionmark + column   try transaction.executesql("insert customer(name address city country values + questionmark + + column   document.write(column look exactly   data[0 data[1 data[2 data[3   wrong column thank advance

## <font color = 'blue'> Create Vocab

In [ ]:
def create_vocab(dataset, min_freq):
  counter = Counter()
  for (text, _) in dataset:
    counter.update(str(text).split())
  my_vocab = vocab(counter, min_freq=min_freq)
  my_vocab.insert_token('<unk>', 0)
  my_vocab.set_default_index(0)
  return my_vocab

<font color = 'green'> vocab should always be created based on trainset

In [ ]:
stack_vocab = create_vocab(trainset, min_freq = 2)

In [ ]:
len(stack_vocab)

14834

In [ ]:
stack_vocab.get_itos()[0:5]

['<unk>', 'comic_strip', 'website', 'like', 'track']

In [ ]:
stack_vocab['abracadabra']

0

## <font color = 'blue'> Collate_fn for Data Loaders

In [ ]:
# Creating a lambda function objects that will be used to get the indices of words from vocab
text_pipeline = lambda x: [stack_vocab[token] for token in str(x).split()]
label_pipeline = lambda x: [int(i) for i in x]

In [ ]:
'''
We know that input to the embedding layers are indices of words from the vocab.
The collate_batch() accepts batch of data and gets the indices of text from vocab and returns the same
We will include this collate_batch() in collat_fn attribute of DataLoader.
So it will create a batch of data containing indices of words and corresponding labels.
But for EmbeddingBag we need one more extra parameter, that is offset.
offsets determines the starting index position of each bag (sequence) in input.
'''
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_text, _label) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return text_list, label_list, offsets

## <font color = 'blue'> **Check Data Loader**



In [ ]:
batch_size=2
check_loader= torch.utils.data.DataLoader(dataset=trainset,
                                        batch_size=batch_size,
                                        shuffle=True,
                                        collate_fn=collate_batch,
                                        num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
for text, label, offsets in check_loader:
  print(label, text, offsets)
  break

tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 1, 0, 0, 1, 0, 0, 0, 0, 0]]) tensor([ 941,  584,   61,  416,   21, 1456,   99,  106,  416, 3364,  643,   89,
          18,  643, 2370, 2371,   22,   18, 4420, 2373, 1272, 2374,  494,  494,
        4421,  494, 4422,  494, 4423,   22,   18, 4420,  494,  646,  494,  647,
         494, 4423,  494,    0,   22,   18, 1612,  494,  646,  494, 4424,  494,
           0,  494,    0,   22,   18, 4425,   22,   18, 4420,  494,  646,  494,
         647,  494, 4423,  494,    0,   22,   18, 4426,  494,  646,  494,  647,
         494,    0,   22,   18, 4420,  494,  646,  494,  647,  494, 4423,   22,
          18, 1612,  494,  646,  494,  647,  494, 4427,  494, 4428,   22,   18,
        4429,   22,   18, 1612,  494,  646,  494,  647,  494, 4427,  494, 4428,
          22,   18, 4429,   22,   18, 1612,  494,  646,  494,  647,  494, 4427,
         494, 4428,   22,   18, 4429,   22,   18, 1612,  494,  646,  494,  647,
         494, 4427,  494, 4428,   22,  

# <font color = 'blue'>Weight Matrix of Pretrained Weights

In [ ]:
gensim.__version__

'4.2.0'

In [ ]:
import gensim.downloader as api
word_vectors = api.load("glove-wiki-gigaword-100")

[==================================================] 100.0% 128.1/128.1MB downloaded


In [ ]:
#pretrained_vectors = KeyedVectors.load('/content/drive/MyDrive/HW 6 B/model_imdb_cbow.bin')
pretrained_vectors = word_vectors

In [ ]:
vector = pretrained_vectors.get_vector('office', norm=True)
vector.shape

(100,)

In [ ]:
# embedding_dim =300
# test_weights = np.zeros((2, embedding_dim))

embedding_dim =100
test_weights = np.zeros((2, embedding_dim))

In [ ]:
test_weights[0] = pretrained_vectors.get_vector('office', norm=True)

In [ ]:
test_weights[1] =  np.random.normal(size=(embedding_dim, ))

In [ ]:
test_weights

array([[-0.01455544, -0.13056442,  0.06381373, -0.00747831,  0.10621653,
         0.02454428, -0.08777763,  0.1584893 ,  0.0725054 ,  0.08593655,
         0.01865983, -0.11025991,  0.05561126,  0.13550204, -0.03868205,
        -0.02299953,  0.10512065,  0.07091681, -0.09158254, -0.07538801,
         0.01319987,  0.04951464,  0.00256787, -0.05680533, -0.12988587,
        -0.14037128,  0.04161026, -0.08634334, -0.0571718 , -0.03574333,
         0.16420542,  0.03900468,  0.0447173 ,  0.10741586, -0.08657654,
         0.14740245, -0.10056353,  0.00941039,  0.05244635, -0.03778255,
        -0.03911339, -0.10175234,  0.06475005, -0.06024378,  0.05340021,
        -0.02505102, -0.06679453, -0.09416356,  0.02800201, -0.07653475,
         0.07485498, -0.18018079,  0.06768527,  0.18184651, -0.03278357,
        -0.4376871 , -0.06637722,  0.02913998,  0.34244165,  0.08327136,
        -0.08592603, -0.03643593, -0.00584571,  0.04131393,  0.03244867,
        -0.07346101,  0.08773556,  0.07319625,  0.0

In [ ]:
len(stack_vocab)

14834

In [ ]:
#embedding_dim = 300
embedding_dim = 100
pretrained_weights = np.zeros((len(stack_vocab), embedding_dim))
words_found = 0
words_not_found = 0

for i, word in enumerate(stack_vocab.get_itos()):
    try: 
        pretrained_weights[i] = pretrained_vectors.get_vector(word, norm=True)
        words_found += 1
    except KeyError:
        words_not_found  += 1
        pretrained_weights[i] = np.random.normal(size=(embedding_dim, ))

In [ ]:
words_found

4196

In [ ]:
words_not_found

10638

# <font color = 'blue'> **Model for ANN**


In [ ]:
class MLPCustom(nn.Module):
  def __init__(self, embed_dim, vocab_size, h_sizes_list, d_prob_list, output_dim, non_linearity, batch_norm, task, pretrained_weights):

    super().__init__()

    self.output_dim = output_dim
    self.vocab_size = vocab_size
    self.embed_dim = embed_dim
    self.h_sizes_list = h_sizes_list
    self.d_prob_list = d_prob_list
    self.batch_norm = batch_norm
    self.task = task
    self.pretrained_weights= pretrained_weights
    
    self.non_linearity = non_linearity

    model_layers = []

    # embedding_layer
    if self.task == 2:
       self.embedding = nn.EmbeddingBag(vocab_size, self.embed_dim)

    # Task 5
    if self.task == 5:
       self.embedding = nn.EmbeddingBag(vocab_size, self.embed_dim).from_pretrained(pretrained_weights,
                                                                               freeze = True)
       
    # Task 6
    if self.task == 6:
       self.embedding = nn.EmbeddingBag(vocab_size, self.embed_dim).from_pretrained(pretrained_weights,
                                                                               freeze = False)

    input_dim = self.embed_dim
    # hidden layers, droput, non_linearity, batchnorm layers
    for k, hidden_size in enumerate(self.h_sizes_list):
      # hidden_layer
      model_layers.append(nn.Linear(input_dim, hidden_size))
      # Activation function
      model_layers.append(self.non_linearity)
      # Dropout Layer
      model_layers.append(nn.Dropout(p=self.d_prob_list[k]))
      # Batch_Norm Layer
      if self.batch_norm:
        model_layers.append(nn.BatchNorm1d(hidden_size, momentum = 0.9))
      input_dim = hidden_size

    # output layer  
    if len(self.h_sizes_list)>0:
        model_layers.append(nn.Linear(self.h_sizes_list[-1], self.output_dim))
    else:
        model_layers.append(nn.Linear(self.embed_dim, self.output_dim))

    self.module_list = nn.ModuleList(model_layers)
    

  def forward(self, x, offsets):
    out = self.embedding(x, offsets) # batchsize, embedding_dim
    for layer in self.module_list:
      out = layer(out)
    return out

    # Note : We do not need to apply softmax as we will be using nn.CrossEntropy Loss

## <Font color = 'pickle'>**Function for Training  Loops**

**Model Training** involves five steps: 

- Step 0: Randomly initialize parameters / weights
- Step 1: Compute model's predictions - forward pass
- Step 2: Compute loss
- Step 3: Compute the gradients
- Step 4: Update the parameters
- Step 5: Repeat steps 1 - 4

Model training is repeating this process over and over, for many **epochs**.

We will specify number of ***epochs*** and during each epoch we will iterate over the complete dataset and will keep on updating the parameters.

***Learning rate*** and ***epochs*** are known as hyperparameters. We have to adjust the values of these two based on validation dataset.

We will now create functions for step 1 to 4.

In [ ]:
def train(train_loader, loss_function, model, optimizer, grad_clipping, max_norm, log_batch, log_interval):

  # Training Loop 

  # initilalize variables as global
  # these counts will be updated every epoch
  global batch_ct_train

  # Initialize train_loss at the he start of the epoch
  running_train_loss = 0
  running_train_correct = 0
  
  # put the model in training mode

  model.train()
  # Iterate on batches from the dataset using train_loader
  for input_, targets, offsets in train_loader:
    
    # move inputs and outputs to GPUs
    input_ = input_.to(device)
    targets = targets.to(device)
    offsets = offsets.to(device)


    # Step 1: Forward Pass: Compute model's predictions 
    output = model(input_, offsets)
    
    # Step 2: Compute loss
    loss = loss_function(output, targets.float())

    # Correct prediction
    y_pred = torch.argmax(output, dim = 1)
    #correct = torch.sum(y_pred == targets)

    batch_ct_train += 1

    # Step 3: Backward pass -Compute the gradients
    optimizer.zero_grad()
    loss.backward()

    # Gradient Clipping
    if grad_clipping:
      nn.utils.clip_grad_norm_(model.parameters(), max_norm=max_norm, norm_type=2)

    # Step 4: Update the parameters
    optimizer.step()
          
    # Add train loss of a batch 
    running_train_loss += loss.item()

    # Add Corect counts of a batch
    #running_train_correct += correct

    # log batch loss and accuracy
    if log_batch:
      if ((batch_ct_train + 1) % log_interval) == 0:
        wandb.log({f"Train Batch Loss  :": loss})
        #wandb.log({f"Train Batch Acc :": correct/len(targets)})

  
  # Calculate mean train loss for the whole dataset for a particular epoch
  train_loss = running_train_loss/len(train_loader)

  # Calculate accuracy for the whole dataset for a particular epoch
  #train_acc = running_train_correct/len(train_loader.dataset)
  

  return train_loss
  #, train_acc

## <Font color = 'pickle'>**Function for Validation Loops**


In [ ]:
def validate(valid_loader, loss_function, model, log_batch, log_interval):

  # initilalize variables as global
  # these counts will be updated every epoch
  global batch_ct_valid

  # Validation/Test loop
  # Initialize valid_loss at the he strat of the epoch
  running_val_loss = 0
  running_val_correct = 0

  # put the model in evaluation mode
  model.eval()

  with torch.no_grad():
    for input_, targets, offsets in valid_loader:

      # move inputs and outputs to GPUs
      input_ = input_.to(device)
      targets = targets.to(device)
      offsets = offsets.to(device)

      # Step 1: Forward Pass: Compute model's predictions 
      output = model(input_, offsets)

      # Step 2: Compute loss
      loss = loss_function(output, targets.float())

      # Correct Predictions
      y_pred = torch.argmax(output, dim = 1)
      #correct = torch.sum(y_pred == targets)

      batch_ct_valid += 1

      # Add val loss of a batch 
      running_val_loss += loss.item()

      # Add correct count for each batch
      #running_val_correct += correct

      # log batch loss and accuracy
      if log_batch:
        if ((batch_ct_valid + 1) % log_interval) == 0:
          wandb.log({f"Valid Batch Loss  :": loss})
          #wandb.log({f"Valid Batch Accuracy :": correct/len(targets)})

    # Calculate mean val loss for the whole dataset for a particular epoch
    val_loss = running_val_loss/len(valid_loader)

    # Calculate accuracy for the whole dataset for a particular epoch
    #val_acc = running_val_correct/len(valid_loader.dataset)

    # scheduler step
    scheduler.step(val_loss)
    # scheduler.step()
    
  return val_loss
  #, val_acc

## <Font color = 'pickle'>**Function for Model Training**
    
We will now create a function for step 5 of model training


In [ ]:
def train_loop(train_loader, valid_loader, model, optimizer, loss_function, epochs, device, patience, early_stopping,
               file_model, save_best_model):
    
  """ 
  Function for training the model and plotting the graph for train & validation loss vs epoch.
  Input: iterator for train dataset, initial weights and bias, epochs, learning rate, batch size.
  Output: final weights, bias and train loss and validation loss for each epoch.
  """

  # Create lists to store train and val loss at each epoch
  train_loss_history = []
  valid_loss_history = []
  train_acc_history = []
  valid_acc_history = []

  # initialize variables for early stopping

  delta = 0
  best_score = None
  valid_loss_min = np.Inf
  counter_early_stop=0
  early_stop=False

  # Iterate for the given number of epochs
  # Step 5: Repeat steps 1 - 4

  for epoch in range(epochs):

    t0 = datetime.now()

    # Get train loss and accuracy for one epoch
    train_loss = train(train_loader, loss_function, model, optimizer, 
                                  wandb.config.GRAD_CLIPPING, wandb.config.MAX_NORM,
                                  wandb.config.LOG_BATCH, wandb.config.LOG_INTERVAL)
    valid_loss = validate(valid_loader, loss_function, model, 
                                       wandb.config.LOG_BATCH, wandb.config.LOG_INTERVAL)

    dt = datetime.now() - t0

    # Save history of the Losses and accuracy
    train_loss_history.append(train_loss)
    #train_acc_history.append(train_acc)

    valid_loss_history.append(valid_loss)
    #valid_acc_history.append(valid_acc)

    # Log the train and valid loss to wandb
    wandb.log({f"Train Loss :": train_loss, "epoch": epoch})
   # wandb.log({f"Train Acc :": train_acc, "epoch": epoch})

    wandb.log({f"Valid Loss :": valid_loss, "epoch": epoch})
    #wandb.log({f"Valid Acc :": valid_acc, "epoch": epoch})

    if early_stopping:
      score = -valid_loss
      if best_score is None:
        best_score=score
        print(f'Validation loss has decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Saving Model...')
        torch.save(model.state_dict(), file_model)
        valid_loss_min = valid_loss

      elif score < best_score + delta:
        counter_early_stop += 1
        print(f'Early stoping counter: {counter_early_stop} out of {patience}')
        if counter_early_stop > patience:
          early_stop = True


      else:
        best_score = score
        print(f'Validation loss has decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Saving model...')
        torch.save(model.state_dict(), file_model)
        counter_early_stop=0
        valid_loss_min = valid_loss

      if early_stop:
        print('Early Stopping')
        break

    elif save_best_model:

      score = -valid_loss
      if best_score is None:
        best_score=score
        print(f'Validation loss has decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Saving Model...')
        torch.save(model.state_dict(), file_model)
        valid_loss_min = valid_loss

      elif score < best_score + delta:
        print(f'Validation loss has not decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Not Saving Model...')
      
      else:
        best_score = score
        print(f'Validation loss has decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Saving model...')
        torch.save(model.state_dict(), file_model)
        valid_loss_min = valid_loss
        
    else:
        torch.save(model.state_dict(), file_model)
    
    # Print the train loss and accuracy for given number of epochs, batch size and number of samples
    print(f'Epoch : {epoch+1} / {epochs}')
    print(f'Time to complete {epoch+1} is {dt}')
    print(f'Learning rate: {scheduler._last_lr[0]}')
    print(f'Train Loss: {train_loss : .4f} %')
    print(f'Valid Loss: {valid_loss : .4f} %')
    print()
    torch.cuda.empty_cache()

  return train_loss_history, valid_loss_history

## <Font color = 'pickle'>**Function for Accuracy and Predictions**

Now we have final values for weights and bias after training the model. We will use these values to make predictions on the test dataset.

In [ ]:
def get_acc_pred(data_loader, model, device):
    
  """ 
  Function to get predictions and accuracy for a given data using estimated model
  Input: Data iterator, Final estimated weoights, bias
  Output: Prections and Accuracy for given dataset
  """

  # Array to store predicted labels
  predictions = torch.Tensor() # empty tensor
  predictions = predictions.to(device) # move predictions to GPU

  # Array to store actual labels
  y = torch.Tensor() # empty tensor
  y = y.to(device)

  # put the model in evaluation mode
  model.eval()
  
  # Iterate over batches from data iterator
  with torch.no_grad():
    for input_, targets, offsets in data_loader:
      
      # move inputs and outputs to GPUs
      
      input_ = input_.to(device)
      targets = targets.to(device)
      offsets = offsets.to(device)
      
      # Calculated the predicted labels
      output = model(input_, offsets)

      # Choose the label 
      predicted_y = output.clone()
  
      # Update teh output 
      predicted_y[predicted_y>0.5] = 1
      predicted_y[predicted_y<=0.5] =0

      # Add the predicted labels to the array
      predictions = torch.cat((predictions, predicted_y)) 

      # Add the actual labels to the array
      y = torch.cat((y, targets)) 

  # Return tuple containing predictions and accuracy
  return predictions, y  

# <Font color = 'pickle'>**Meta Data**

In [ ]:
hyperparameters = SimpleNamespace(
    EMBED_DIM = 100, # CHANGED FROM 300 TO 100
    VOCAB_SIZE = len(stack_vocab),
    OUTPUT_DIM = 10,
    HIDDEN_SIZES_LIST = [200], # 100 layers of size 200  [200]*100
    DPROB_LIST = [0],
    NON_LINEARITY= nn.SELU(),
    PRETRAINED_WEIGHTS_TENSOR = torch.tensor(pretrained_weights).float(),
    BATCH_NORM = True,
    EPOCHS = 30,
    TASK = 6,    
    BATCH_SIZE = 256,
    LEARNING_RATE = 0.1,
    DATASET="IMDB",
    ARCHITECTUREe="nn_with_pre_trained_embeddings",
    LOG_INTERVAL = 25,
    LOG_BATCH = True,
    FILE_MODEL = '/content/drive/MyDrive/HW 6 B/hw6_part_B_task_5.pt',
    GRAD_CLIPPING = False,
    MAX_NORM = 0,
    MOMENTUM = 0,
    PATIENCE = 5,
    EARLY_STOPPING = True,
    SCHEDULER_FACTOR = 0,
    SCHEDULER_PATIENCE = 0,
    WEIGHT_DECAY = 0.002,
    SAVE_BEST_MODEL = False,
    DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    )

# <Font color = 'pickle'>**Data Loaders, Loss Function, Optimizer**

In [ ]:
# Initialize a new project
import random
wandb.init(name = 'task2', project = 'NLP_imdb_pre_train_embed', config = hyperparameters)

wandb: Currently logged in as: teamnine. Use `wandb login --relogin` to force relogin


In [ ]:
wandb.config = hyperparameters
wandb.config

namespace(ARCHITECTUREe='nn_with_pre_trained_embeddings', BATCH_NORM=True, BATCH_SIZE=256, DATASET='IMDB', DEVICE=device(type='cuda', index=0), DPROB_LIST=[0], EARLY_STOPPING=True, EMBED_DIM=100, EPOCHS=30, FILE_MODEL='/content/drive/MyDrive/HW 6 B/hw6_part_B_task_5.pt', GRAD_CLIPPING=False, HIDDEN_SIZES_LIST=[200], LEARNING_RATE=0.1, LOG_BATCH=True, LOG_INTERVAL=25, MAX_NORM=0, MOMENTUM=0, NON_LINEARITY=SELU(), OUTPUT_DIM=10, PATIENCE=5, PRETRAINED_WEIGHTS_TENSOR=tensor([[ 1.8832, -1.3478, -1.2705,  ...,  0.8235,  2.1632,  1.3365],
        [-0.3692, -0.2394,  1.0997,  ...,  0.5830, -0.3994,  0.3701],
        [-0.0430, -0.0854, -0.0288,  ..., -0.0470,  0.1472,  0.0645],
        ...,
        [ 1.4744, -1.3274, -0.2250,  ..., -0.4070, -1.3672, -0.5155],
        [-1.1898, -0.8310,  0.4049,  ..., -0.7881, -0.1413,  0.4693],
        [ 0.0723,  0.0390,  0.0493,  ..., -0.0819, -0.1261, -0.0316]]), SAVE_BEST_MODEL=False, SCHEDULER_FACTOR=0, SCHEDULER_PATIENCE=0, TASK=6, VOCAB_SIZE=14834, WEIGH

In [ ]:
# Fix seed value
SEED = 2345
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# Data Loader
train_loader = torch.utils.data.DataLoader(trainset, batch_size=wandb.config.BATCH_SIZE, shuffle = True, 
                                           collate_fn=collate_batch, num_workers = 4)
valid_loader = torch.utils.data.DataLoader(validset, batch_size=wandb.config.BATCH_SIZE, shuffle = False, 
                                           collate_fn=collate_batch,  num_workers = 4)
test_loader = torch.utils.data.DataLoader(testset, batch_size=wandb.config.BATCH_SIZE,   shuffle = False, 
                                          collate_fn=collate_batch,  num_workers = 4)

# cross entropy loss function
loss_function = nn.BCEWithLogitsLoss()

# model 
model_stack = MLPCustom(wandb.config.EMBED_DIM, 
                       wandb.config.VOCAB_SIZE, 
                       wandb.config.HIDDEN_SIZES_LIST, 
                       wandb.config.DPROB_LIST,
                       wandb.config.OUTPUT_DIM, 
                       wandb.config.NON_LINEARITY,
                       wandb.config.BATCH_NORM,
                       wandb.config.TASK,
                       wandb.config.PRETRAINED_WEIGHTS_TENSOR)

model_stack.to(wandb.config.DEVICE)

def init_weights(m):
  if type(m) == nn.Linear:
      torch.nn.init.kaiming_normal_(m.weight)
      torch.nn.init.zeros_(m.bias)
        
# apply initialization recursively  to all modules
# model_imdb.apply(init_weights)

# Intialize stochiastic gradient descent optimizer
optimizer = torch.optim.SGD(model_stack.parameters(), 
                             lr = wandb.config.LEARNING_RATE, 
                             weight_decay=wandb.config.WEIGHT_DECAY)

# wandb.config.OPTIMIZER = optimizer

scheduler = ReduceLROnPlateau(optimizer, mode='min', factor= wandb.config.SCHEDULER_FACTOR, 
                              patience=wandb.config.SCHEDULER_PATIENCE, verbose=True)

#scheduler = StepLR(optimizer, gamma=0.4,step_size=1, verbose=True)

In [ ]:
wandb.config

namespace(ARCHITECTUREe='nn_with_pre_trained_embeddings', BATCH_NORM=True, BATCH_SIZE=256, DATASET='IMDB', DEVICE=device(type='cuda', index=0), DPROB_LIST=[0], EARLY_STOPPING=True, EMBED_DIM=100, EPOCHS=30, FILE_MODEL='/content/drive/MyDrive/HW 6 B/hw6_part_B_task_5.pt', GRAD_CLIPPING=False, HIDDEN_SIZES_LIST=[200], LEARNING_RATE=0.1, LOG_BATCH=True, LOG_INTERVAL=25, MAX_NORM=0, MOMENTUM=0, NON_LINEARITY=SELU(), OUTPUT_DIM=10, PATIENCE=5, PRETRAINED_WEIGHTS_TENSOR=tensor([[ 1.8832, -1.3478, -1.2705,  ...,  0.8235,  2.1632,  1.3365],
        [-0.3692, -0.2394,  1.0997,  ...,  0.5830, -0.3994,  0.3701],
        [-0.0430, -0.0854, -0.0288,  ..., -0.0470,  0.1472,  0.0645],
        ...,
        [ 1.4744, -1.3274, -0.2250,  ..., -0.4070, -1.3672, -0.5155],
        [-1.1898, -0.8310,  0.4049,  ..., -0.7881, -0.1413,  0.4693],
        [ 0.0723,  0.0390,  0.0493,  ..., -0.0819, -0.1261, -0.0316]]), SAVE_BEST_MODEL=False, SCHEDULER_FACTOR=0, SCHEDULER_PATIENCE=0, TASK=6, VOCAB_SIZE=14834, WEIGH

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# <Font color = 'pickle'>**Training Model**

In [ ]:
wandb.watch(model_stack, log = 'all', log_freq=25, log_graph=True)

wandb: logging graph, to disable use `wandb.watch(log_graph=False)`


In [ ]:
# See live graphs in the notebook.
#%%wandb 
batch_ct_train, batch_ct_valid = 0, 0
train_loss_history, valid_loss_history = train_loop(train_loader, 
                                                                                          valid_loader, 
                                                                                          model_stack, 
                                                                                          optimizer,
                                                                                          loss_function, 
                                                                                          wandb.config.EPOCHS, 
                                                                                          wandb.config.DEVICE,
                                                                                          wandb.config.PATIENCE, 
                                                                                          wandb.config.EARLY_STOPPING,
                                                                                          wandb.config.FILE_MODEL,
                                                                                          wandb.config.SAVE_BEST_MODEL)

Validation loss has decreased (inf --> 0.678027). Saving Model...
Epoch : 1 / 30
Time to complete 1 is 0:00:04.065911
Learning rate: 0.1
Train Loss:  0.7081 %
Valid Loss:  0.6780 %

Validation loss has decreased (0.678027 --> 0.661214). Saving model...
Epoch : 2 / 30
Time to complete 2 is 0:00:01.487604
Learning rate: 0.1
Train Loss:  0.6779 %
Valid Loss:  0.6612 %

Validation loss has decreased (0.661214 --> 0.642783). Saving model...
Epoch : 3 / 30
Time to complete 3 is 0:00:01.547232
Learning rate: 0.1
Train Loss:  0.6541 %
Valid Loss:  0.6428 %

Validation loss has decreased (0.642783 --> 0.618518). Saving model...
Epoch : 4 / 30
Time to complete 4 is 0:00:01.575871
Learning rate: 0.1
Train Loss:  0.6329 %
Valid Loss:  0.6185 %

Validation loss has decreased (0.618518 --> 0.604254). Saving model...
Epoch : 5 / 30
Time to complete 5 is 0:00:01.537205
Learning rate: 0.1
Train Loss:  0.6129 %
Valid Loss:  0.6043 %

Validation loss has decreased (0.604254 --> 0.592164). Saving model...

# <Font color = 'pickle'>**Get Accuracy, Predictions**

In [ ]:
device

device(type='cuda', index=0)

In [ ]:
model_nn = MLPCustom(wandb.config.EMBED_DIM, 
                       wandb.config.VOCAB_SIZE, 
                       wandb.config.HIDDEN_SIZES_LIST, 
                       wandb.config.DPROB_LIST,
                       wandb.config.OUTPUT_DIM, 
                       wandb.config.NON_LINEARITY,
                       wandb.config.BATCH_NORM,
                       wandb.config.TASK,
                       wandb.config.PRETRAINED_WEIGHTS_TENSOR)
model_nn.to(device)
model_nn.load_state_dict(torch.load(wandb.config.FILE_MODEL))

<All keys matched successfully>

In [ ]:
# Get the prediction and labels
train_pred,  train_label = get_acc_pred(train_loader, model_nn,device)
valid_pred, valid_label = get_acc_pred(valid_loader, model_nn,device)
test_pred,  test_label  = get_acc_pred(test_loader, model_nn,device)

In [ ]:
pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 529 kB 7.7 MB/s 


In [ ]:
from torchmetrics import F1Score
f1score  = F1Score(num_classes=10, mdmc_average= 'global').to(device)
train_f1_score = f1score( train_pred, train_label.long())
train_f1_score

tensor(0.1183, device='cuda:0')

In [ ]:
# convert these to numpy array
y_train, y_predicted_train  = train_label.cpu().numpy(), train_pred.cpu().numpy() 
y_valid, y_predicted_valid  = valid_label.cpu().numpy(), valid_pred.cpu().numpy() 
y_test, y_predicted_test  = test_label.cpu().numpy(), test_pred.cpu().numpy() 

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
f1_score_train = f1_score(y_train, y_predicted_train, average = 'micro')
f1_score_valid = f1_score(y_valid, y_predicted_valid, average = 'micro')
f1_score_test = f1_score(y_test, y_predicted_test, average = 'micro')

In [ ]:
# Print Accuracy based on saved Model
print('f1_score_train', f1_score_train)
print('f1_score_valid', f1_score_valid)
print('f1_score_test', f1_score_test)

f1_score_train 0.11825845940639795
f1_score_valid 0.04089581304771178
f1_score_test 0.10342574442266404


In [ ]:
wandb.log({'Train f1 score': f1_score_train})
wandb.log({'Valid f1 score': f1_score_valid}) 
wandb.log({'Test f1 score': f1_score_test})

In [ ]:
wandb.finish() 

Test f1 score,▁
Train Batch Loss :,█▇▆▅▅▄▃▃▂▂▂▁▁▁▁▁
Train Loss :,█▇▇▆▆▅▅▄▄▄▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁
Train f1 score,▁
Valid Batch Loss :,█▃▁▂
Valid Loss :,██▇▆▆▆▅▄▄▄▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁
Valid f1 score,▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
Test f1 score,0.10343
Train Batch Loss :,0.42068
Train Loss :,0.42337
